In [1]:
include("LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra

Load spin-boson data and simulate LME data 

In [32]:
data_dir = "DATA/"

γ_list = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]
γᵢ = γ_list[2]

γᵢ = string(γᵢ)

if γᵢ == "0.0" 
    ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", "0.079477")
    ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", "0.079477")
    ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", "0.079477")
    ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", "0.079477")
else 
    ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵢ)
    ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵢ)
    ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵢ)
    ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵢ)
end

ωᶠ = 25.126

Hᴸᴹᴱ = [ ωᶠ        0
         0        0   ]

γᶠ = parse(Float64, γᵢ)

Aᴸᴹᴱ = [ 0    √γᶠ
         0     0. + 0im  ]

tᵉᶠ = convert.(Float64, tᵉ); tᵍᶠ = convert.(Float64, tᵍ); 
tˣᶠ = convert.(Float64, tˣ); tʸᶠ = convert.(Float64, tʸ); 

t = tˣᶠ
Δt = t[2] - t[1] #0.02

ρᵉᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρᵉ[1], tᵉᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
ρᵍᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρᵍ[1], tᵍᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
ρˣᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρˣ[1], tˣᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
ρʸᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρʸ[1], tʸᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ]);


Perform DMD

In [33]:
Aᴰᴹᴰ⁻ˢᴮ = LiPoSID.direct_DMD_01XY_b4_A([ρᵉ, ρᵍ, ρˣ, ρʸ])
Aᴰᴹᴰ⁻ᴸᴹᴱ = LiPoSID.direct_DMD_01XY_b4_A([ρᵉᴸᴹᴱ, ρᵍᴸᴹᴱ, ρˣᴸᴹᴱ, ρʸᴸᴹᴱ])

4×4 Matrix{Float64}:
 0.985178     -0.166585     -9.72695e-12  9.6833e-12
 0.166585      0.985178     -1.40108e-10  1.32683e-10
 2.03323e-13  -3.89423e-13   0.998326     0.00167413
 2.00361e-16   5.64652e-16  -1.38778e-15  1.0

QPT on one DMD step to find Kraus operator-sum representation

In [34]:
ρᵍ₀ = [1. 0]' * [1 0]
ρᵉ₀ = [0. 1]' * [0 1]
ρˣ₀ = [1  1]' * [1 1]/2
ρʸ₀ = [1 -im]' * [1 -im]/2

ρᵍ₁ᴸᴹᴱ = LiPoSID.DMD_step(Aᴰᴹᴰ⁻ᴸᴹᴱ, ρᵍ₀)
ρᵉ₁ᴸᴹᴱ = LiPoSID.DMD_step(Aᴰᴹᴰ⁻ᴸᴹᴱ, ρᵉ₀)
ρˣ₁ᴸᴹᴱ = LiPoSID.DMD_step(Aᴰᴹᴰ⁻ᴸᴹᴱ, ρˣ₀)
ρʸ₁ᴸᴹᴱ = LiPoSID.DMD_step(Aᴰᴹᴰ⁻ᴸᴹᴱ, ρʸ₀)

ρᵍ₁ˢᴮ = LiPoSID.DMD_step(Aᴰᴹᴰ⁻ˢᴮ, ρᵍ₀)
ρᵉ₁ˢᴮ = LiPoSID.DMD_step(Aᴰᴹᴰ⁻ˢᴮ, ρᵉ₀)
ρˣ₁ˢᴮ = LiPoSID.DMD_step(Aᴰᴹᴰ⁻ˢᴮ, ρˣ₀)
ρʸ₁ˢᴮ = LiPoSID.DMD_step(Aᴰᴹᴰ⁻ˢᴮ, ρʸ₀)

Eᴰᴹᴰ⁻ᴸᴹᴱ, dᴸᴹᴱ = LiPoSID.QPT(ρᵍ₁ᴸᴹᴱ, ρᵉ₁ᴸᴹᴱ, ρˣ₁ᴸᴹᴱ, ρʸ₁ᴸᴹᴱ)
Eᴰᴹᴰ⁻ˢᴮ, dˢᴮ = LiPoSID.QPT(ρᵍ₁ˢᴮ, ρᵉ₁ˢᴮ, ρˣ₁ˢᴮ, ρʸ₁ˢᴮ)

print(dˢᴮ) # one of eigenvalues is 10⁻⁴ negative for spin-boson data !!!

[-0.0001717271374018512, -8.463316694217566e-7, 0.0010104484285401723, 0.9991621250405306]

In [31]:
dˢᴮ

4-element Vector{Float64}:
 -0.00015396989514781766
  5.546387344563031e-7
  0.0009480763218229412
  0.9992053389345906

POP

In [17]:
using DynamicPolynomials

operators_num = length(Eᴰᴹᴰ⁻ˢᴮ) # 3

@polyvar α[1:2, 1:2, 1:operators_num]
@polyvar β[1:2, 1:2, 1:operators_num]


Kₛ = []

for i in 1:operators_num
    K = α[:,:,i] + im * β[:,:,i]
    push!(Kₛ, K)
end

objᵍ = LiPoSID.kraus_obj(ρᵍ, Kₛ)
objᵉ = LiPoSID.kraus_obj(ρᵉ, Kₛ)
objˣ = LiPoSID.kraus_obj(ρˣ, Kₛ)
objʸ = LiPoSID.kraus_obj(ρʸ, Kₛ)

objˢᴮ =  objᵍ + objᵉ + objˣ + objʸ

objᵍᴸᴹᴱ = LiPoSID.kraus_obj(ρᵍᴸᴹᴱ, Kₛ)
objᵉᴸᴹᴱ = LiPoSID.kraus_obj(ρᵉᴸᴹᴱ, Kₛ)
objˣᴸᴹᴱ = LiPoSID.kraus_obj(ρˣᴸᴹᴱ, Kₛ)
objʸᴸᴹᴱ = LiPoSID.kraus_obj(ρʸᴸᴹᴱ, Kₛ)

objᴸᴹᴱ =  objᵍᴸᴹᴱ + objᵉᴸᴹᴱ + objˣᴸᴹᴱ + objʸᴸᴹᴱ

constr = LiPoSID.frobenius_norm2(sum(k' * k for k in Kₛ) - I);


In [ ]:
function solution_from_operator_sum(E)

    sol = []

    for e in E
        # push α
        push!(sol, real(e[1,1]))
        push!(sol, real(e[2,1]))
        push!(sol, real(e[1,2]))
        push!(sol, real(e[2,2]))
    end

    for e in E
        # push β 
        push!(sol, imag(e[1,1]))
        push!(sol, imag(e[2,1]))
        push!(sol, imag(e[1,2]))
        push!(sol, imag(e[2,2]))
    end

    return convert.(Float64, sol)

end

solᴰᴹᴰ⁻ˢᴮ = solution_from_operator_sum(Eᴰᴹᴰ⁻ˢᴮ)
solᴰᴹᴰ⁻ᴸᴹᴱ = solution_from_operator_sum(Eᴰᴹᴰ⁻ᴸᴹᴱ);

In [29]:
Kₛ[1]

2×2 Matrix{Polynomial{true, Complex{Int64}}}:
 α₁₋₁₋₁ + (0+1im)β₁₋₁₋₁  α₁₋₂₋₁ + (0+1im)β₁₋₂₋₁
 α₂₋₁₋₁ + (0+1im)β₂₋₁₋₁  α₂₋₂₋₁ + (0+1im)β₂₋₂₋₁

In [7]:
using TSSOS

Spin-boson

In [18]:
solutionᴰᴹᴰ⁻ˢᴮ = variables(objˢᴮ) => solᴰᴹᴰ⁻ˢᴮ

opt,solᴾᴼᴾ⁻ˢᴮ,data = tssos_first([objˢᴮ, constr], variables(objˢᴮ), maxdegree(objˢᴮ)÷2, numeq=1, solution=true, QUIET=true); 

solutionᴾᴼᴾ⁻ˢᴮ = variables(objˢᴮ) => solᴾᴼᴾ⁻ˢᴮ

*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...


optimum = 0.00011596498709466529


Global optimality certified with relative optimality gap 0.001696%!


PolyVar{true}[α₁₋₁₋₁, α₂₋₁₋₁, α₁₋₂₋₁, α₂₋₂₋₁, α₁₋₁₋₂, α₂₋₁₋₂, α₁₋₂₋₂, α₂₋₂₋₂, α₁₋₁₋₃, α₂₋₁₋₃  …  β₁₋₂₋₁, β₂₋₂₋₁, β₁₋₁₋₂, β₂₋₁₋₂, β₁₋₂₋₂, β₂₋₂₋₂, β₁₋₁₋₃, β₂₋₁₋₃, β₁₋₂₋₃, β₂₋₂₋₃] => [-0.06817538004756007, -0.002057825298363371, -0.0074352894377520224, -0.01167990827923023, 0.7908952341027069, 0.0008703010664475472, 0.004854955029530062, 0.48733031161564544, 0.2987463656926836, -0.005917473960731478  …  0.02646169693242808, -0.12014318306808237, 0.4261602160349271, 0.0035014358852516565, 0.015256258939052476, 0.7538854664144, -0.29822768890535234, -0.00010300055740795718, 0.0012454042054803277, -0.11730366419144578]

In [19]:
convert(Float64, subs(objˢᴮ, solutionᴾᴼᴾ⁻ˢᴮ))

0.0001329208744209609

In [20]:
convert(Float64, subs(objˢᴮ, solutionᴰᴹᴰ⁻ˢᴮ))

0.0002516249378459179

In [21]:
convert(Float64, subs(objˢᴮ, solutionᴰᴹᴰ⁻ˢᴮ)) > convert(Float64, subs(objˢᴮ, solutionᴾᴼᴾ⁻ˢᴮ))

true

LME

In [22]:
solutionᴰᴹᴰ⁻ᴸᴹᴱ = variables(objᴸᴹᴱ) => solᴰᴹᴰ⁻ᴸᴹᴱ

opt,solᴾᴼᴾ⁻ᴸᴹᴱ,data = tssos_first([objᴸᴹᴱ, constr], variables(objᴸᴹᴱ), maxdegree(objᴸᴹᴱ)÷2, numeq=1, solution=true, QUIET=true);

solutionᴾᴼᴾ⁻ᴸᴹᴱ = variables(objᴸᴹᴱ) => solᴾᴼᴾ⁻ᴸᴹᴱ

*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...


optimum = -5.7741888646504704e-6


Global optimality certified with relative optimality gap 0.000577%!


PolyVar{true}[α₁₋₁₋₁, α₂₋₁₋₁, α₁₋₂₋₁, α₂₋₂₋₁, α₁₋₁₋₂, α₂₋₁₋₂, α₁₋₂₋₂, α₂₋₂₋₂, α₁₋₁₋₃, α₂₋₁₋₃  …  β₁₋₂₋₁, β₂₋₂₋₁, β₁₋₁₋₂, β₂₋₁₋₂, β₁₋₂₋₂, β₂₋₂₋₂, β₁₋₁₋₃, β₂₋₁₋₃, β₁₋₂₋₃, β₂₋₂₋₃] => [-0.4979415273706021, -1.3524607371630385e-6, -0.004145741552723083, -0.7429513764836689, -0.1853736879917971, -4.274500546717768e-7, 0.029163558001829425, -0.00962809339371518, 0.43757894339972847, 9.32971960568954e-6  …  0.01118725724895354, 0.31877882078118014, -0.3172940615799985, -5.990445267638307e-7, 0.02276668493311423, -0.3670892674019886, 0.1360602625367783, -6.4203262551306805e-6, -0.002170180473002704, 0.329745972799668]

In [23]:
convert(Float64, subs(objᴸᴹᴱ, solutionᴾᴼᴾ⁻ᴸᴹᴱ))


7.275957614183426e-12

In [24]:
convert(Float64, subs(objᴸᴹᴱ, solutionᴰᴹᴰ⁻ᴸᴹᴱ))

141.85225456279932

In [25]:
convert(Float64, subs(objᴸᴹᴱ, solutionᴰᴹᴰ⁻ᴸᴹᴱ))> convert(Float64, subs(objᴸᴹᴱ, solutionᴾᴼᴾ⁻ᴸᴹᴱ))

true

In [36]:
subs.(Kₛ, solutionᴰᴹᴰ⁻ᴸᴹᴱ)

3-element Vector{Matrix{Polynomial{true, ComplexF64}}}:
 [(-5.339947039293543e-9 + 0.9686252481984937im) (-1.4359179056818192e-15 - 2.199627432437654e-10im); (5.198124257191697e-6 + 1.1555107812329576e-10im) (5.796336585128487e-9 + 0.9678050392429287im)]
 [(0.00022124843598855925 + 2.308704774895905e-9im) (-6.329804459044095e-11 + 3.896850179382233e-15im); (5.054393346440616e-7 + 3.896850179382233e-15im) (-0.0002214359427087189 + 5.490612301386355e-10im)]
 [(4.555337069469106e-9 - 5.6815106033229354e-5im) (0.03985319821372898 - 3.654153947571599e-11im); (2.433072763111713e-11 + 7.986455662363599e-8im) (-6.135817129761322e-9 - 5.6815106033229354e-5im)]

In [38]:
[convert.(ComplexF64, subs(k, solutionᴰᴹᴰ⁻ᴸᴹᴱ)) for k in Kₛ]

3-element Vector{Matrix{ComplexF64}}:
 [-5.339947039293543e-9 + 0.9686252481984937im -1.4359179056818192e-15 - 2.199627432437654e-10im; 5.198124257191697e-6 + 1.1555107812329576e-10im 5.796336585128487e-9 + 0.9678050392429287im]
 [0.00022124843598855925 + 2.308704774895905e-9im -6.329804459044095e-11 + 3.896850179382233e-15im; 5.054393346440616e-7 + 3.896850179382233e-15im -0.0002214359427087189 + 5.490612301386355e-10im]
 [4.555337069469106e-9 - 5.6815106033229354e-5im 0.03985319821372898 - 3.654153947571599e-11im; 2.433072763111713e-11 + 7.986455662363599e-8im -6.135817129761322e-9 - 5.6815106033229354e-5im]